In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
train_id = train['Id']
test_id = test['Id']
train = train.drop(['Id'], axis=1)
test = test.drop(['Id'], axis=1)

In [ ]:
train.shape

(1460, 80)

In [ ]:
test.shape

(1459, 79)

In [ ]:
train.corrwith(train['SalePrice']).abs().sort_values()[::-1]

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
KitchenAbvGr     0.135907
EnclosedPorch    0.128578
ScreenPorch      0.111447
PoolArea         0.092404
MSSubClass       0.084284
OverallCond      0.077856
MoSold           0.046432
3SsnPorch        0.044584
YrSold           0.028923
LowQualFinSF     0.025606
MiscVal          0.021190
BsmtHalfBath     0.016844
BsmtFinSF2       0.011378
dtype: float64

In [ ]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]

In [ ]:
train_prices = train.SalePrice.values
train["SalePrice"] = np.log1p(train["SalePrice"])
y_train = train.SalePrice.values

In [ ]:
all = pd.concat((train, test)).reset_index(drop=True)
all.drop(['SalePrice'], axis=1, inplace=True)

In [ ]:
total_missings=missing_data(all)
total_missings_train.head(15)

,Total,Percent
PoolQC,1453,99.520548
MiscFeature,1406,96.301370
Alley,1369,93.767123
Fence,1179,80.753425
FireplaceQu,690,47.260274
LotFrontage,259,17.739726
GarageCond,81,5.547945
GarageType,81,5.547945
GarageYrBlt,81,5.547945
GarageFinish,81,5.547945


In [ ]:
all = all.drop((total_missings[total_missings['Percent'] > 80]).index,1)

In [ ]:
all = pd.get_dummies(all)

In [ ]:
numeric = all.select_dtypes(exclude = ["object"]).columns
categorical = all.select_dtypes(include = ["object"]).columns

In [ ]:
missings = [] 

for i in categorical:
  missings.append(all[i].value_counts().idxmax())

for i in range(len(categorical)):
    all[categorical[i]].fillna(missings[i], inplace=True)

for i in numeric:
    all[i].fillna(all[i].median(), inplace=True)

In [ ]:
train = all[:ntrain]
test = all[ntrain:]

In [ ]:
kf = KFold(n_splits=5)
kf.get_n_splits(train)

5

In [ ]:
print(kf)
rmse_list = []
for alpha in [0.001, 0.01, 0.1, 1, 10]:
  model = Lasso(alpha=alpha)
  for rand_state in [10,20,30,42,50]:
      X_train, X_test, y_train_tts, y_test_tts = train_test_split(train, y_train, test_size=0.25, random_state=rand_state)
      model.fit(X_train, y_train_tts)
      predict = model.predict(X_test)
      rmse = np.sqrt(mean_squared_error(predict,y_test_tts))
      rmse_list.append(rmse)
      
  print("Mean rmse for alpha={}: {}".format(alpha, str(np.mean(rmse_list))))
  print("std rmse for alpha={}: {}".format(alpha, str(np.std(rmse_list))))

KFold(n_splits=5, random_state=None, shuffle=False)
Mean rmse for alpha=0.001: 0.12420558054332989
std rmse for alpha=0.001: 0.006430745519092559
Mean rmse for alpha=0.01: 0.13326800181718412
std rmse for alpha=0.01: 0.010835708833893521
Mean rmse for alpha=0.1: 0.14594170768933673
std rmse for alpha=0.1: 0.02055315827742867
Mean rmse for alpha=1: 0.15482039602618175
std rmse for alpha=1: 0.02391747587022895
Mean rmse for alpha=10: 0.16854938704263986
std rmse for alpha=10: 0.03492794856798202


In [ ]:
model = Lasso(alpha=0.001)
model.fit(train, y_train)
preds = model.predict(test)

In [ ]:
submission = pd.DataFrame()
submission['Id'] = test_ID
submission['SalePrice'] = np.exp(preds)
submission.to_csv('sub_test.csv',index=False)

In [ ]:
df = pd.read_csv('sub_test.csv')
df.head(5)

,Id,SalePrice
0,1461,115987.839490
1,1462,139249.980300
2,1463,171020.300787
3,1464,200264.201783
4,1465,197228.497228
